# Renaming EPIs

This notebook renames erroneous EPI files by reordering their names

In [1]:
import flywheel
import re

In [2]:
client = flywheel.Client()
proj = client.projects.find_first('label=RBC_HBN')

In [3]:
subjects = proj.subjects()

In [4]:
subslist = '''sub-NDARAD571FLB
sub-NDARAA306NT2
sub-NDARAM357BUA
sub-NDARAL897CYV
sub-NDARAT709MMX
sub-NDARAG191AE8
sub-NDARAP049KXJ
sub-NDARAP457WB5
sub-NDARAB977GFB
sub-NDARAW216PM7
sub-NDARAW247CCF
sub-NDARAC853CR6
'''.split('\n')

subs = [x for x in subjects if x.label in subslist]

In [5]:
len(subs)

12

In [6]:
for s in subs:
    print(len(s.sessions()))

1
1
1
1
1
1
1
1
1
1
1
1


In [8]:
for s in subs:
    
    sess = s.sessions()[0]
    
    for acq in sess.acquisitions():
        
        fi = acq.files[0]
        
        if 'epi' in fi.name:
            print(fi.name)

sub-NDARAL897CYV_ses-HBNsiteCUNY_dir-AP_acq-fMRI_epi.nii.gz
sub-NDARAL897CYV_ses-HBNsiteCUNY_dir-PA_acq-fMRI_epi.nii.gz
sub-NDARAL897CYV_ses-HBNsiteCUNY_dir-AP_acq-dwi_epi.nii.gz
sub-NDARAL897CYV_ses-HBNsiteCUNY_dir-PA_acq-dwi_epi.nii.gz
sub-NDARAC853CR6_ses-HBNsiteCUNY_dir-AP_acq-fMRI_epi.nii.gz
sub-NDARAC853CR6_ses-HBNsiteCUNY_dir-PA_acq-fMRI_epi.nii.gz
sub-NDARAC853CR6_ses-HBNsiteCUNY_dir-AP_acq-dwi_epi.nii.gz
sub-NDARAC853CR6_ses-HBNsiteCUNY_dir-PA_acq-dwi_epi.nii.gz
sub-NDARAM357BUA_ses-HBNsiteCUNY_dir-AP_acq-fMRI_epi.nii.gz
sub-NDARAM357BUA_ses-HBNsiteCUNY_dir-AP_acq-dwi_epi.nii.gz
sub-NDARAM357BUA_ses-HBNsiteCUNY_dir-PA_acq-dwi_epi.nii.gz
sub-NDARAG191AE8_ses-HBNsiteCBIC_dir-AP_acq-dwi_epi.nii.gz
sub-NDARAG191AE8_ses-HBNsiteCBIC_dir-PA_acq-dwi_epi.nii.gz
sub-NDARAG191AE8_ses-HBNsiteCBIC_dir-PA_acq-fMRI_epi.nii.gz
sub-NDARAG191AE8_ses-HBNsiteCBIC_dir-AP_acq-fMRI_epi.nii.gz
sub-NDARAD571FLB_ses-HBNsiteRU_dir-AP_acq-fMRI_epi.nii.gz
sub-NDARAD571FLB_ses-HBNsiteRU_dir-PA_acq-fMRI_epi

Here's how to move the key value pairs:

In [17]:
fname = 'sub-NDARAW247CCF_ses-HBNsiteRU_dir-PA_acq-dwi_epi.nii.gz'

bids_vals = re.findall(r'[a-z]+-[a-zA-Z0-9]+(?=_)', fname)
print(bids_vals)

dir_index = [i for i, x in enumerate(bids_vals) if 'dir' in x]
print(dir_index)

suffix = re.search(r'[^_]+$', fname).group(0)

bids_vals.append(bids_vals.pop(dir_index[0]))
bids_vals.append(suffix)
print(bids_vals)

final = '_'.join(bids_vals)

print(final)

['sub-NDARAW247CCF', 'ses-HBNsiteRU', 'dir-PA', 'acq-dwi']
[2]
['sub-NDARAW247CCF', 'ses-HBNsiteRU', 'acq-dwi', 'dir-PA', 'epi.nii.gz']
sub-NDARAW247CCF_ses-HBNsiteRU_acq-dwi_dir-PA_epi.nii.gz


The above in a function:

In [20]:
def reorder_kvs(fname):
    
    bids_vals = re.findall(r'[a-z]+-[a-zA-Z0-9]+(?=_)', fname)
    #print(bids_vals)

    dir_index = [i for i, x in enumerate(bids_vals) if 'dir' in x]
    #print(dir_index)

    suffix = re.search(r'[^_]+$', fname).group(0)

    bids_vals.append(bids_vals.pop(dir_index[0]))
    bids_vals.append(suffix)
    #print(bids_vals)

    final = '_'.join(bids_vals)
    
    return final

In [21]:
reorder_kvs('sub-NDARAW247CCF_ses-HBNsiteRU_dir-AP_acq-dwi_epi.nii.gz')

'sub-NDARAW247CCF_ses-HBNsiteRU_acq-dwi_dir-AP_epi.nii.gz'

In [24]:
for s in subs:
    
    sess = s.sessions()[0]
    
    for acq in sess.acquisitions():
        
        fi = acq.files[0]
        
        if 'epi' in fi.name:
            print(fi.name, 'becomes:\n', reorder_kvs(fi.name))
            print()

sub-NDARAL897CYV_ses-HBNsiteCUNY_dir-AP_acq-fMRI_epi.nii.gz becomes:
 sub-NDARAL897CYV_ses-HBNsiteCUNY_acq-fMRI_dir-AP_epi.nii.gz

sub-NDARAL897CYV_ses-HBNsiteCUNY_dir-PA_acq-fMRI_epi.nii.gz becomes:
 sub-NDARAL897CYV_ses-HBNsiteCUNY_acq-fMRI_dir-PA_epi.nii.gz

sub-NDARAL897CYV_ses-HBNsiteCUNY_dir-AP_acq-dwi_epi.nii.gz becomes:
 sub-NDARAL897CYV_ses-HBNsiteCUNY_acq-dwi_dir-AP_epi.nii.gz

sub-NDARAL897CYV_ses-HBNsiteCUNY_dir-PA_acq-dwi_epi.nii.gz becomes:
 sub-NDARAL897CYV_ses-HBNsiteCUNY_acq-dwi_dir-PA_epi.nii.gz

sub-NDARAC853CR6_ses-HBNsiteCUNY_dir-AP_acq-fMRI_epi.nii.gz becomes:
 sub-NDARAC853CR6_ses-HBNsiteCUNY_acq-fMRI_dir-AP_epi.nii.gz

sub-NDARAC853CR6_ses-HBNsiteCUNY_dir-PA_acq-fMRI_epi.nii.gz becomes:
 sub-NDARAC853CR6_ses-HBNsiteCUNY_acq-fMRI_dir-PA_epi.nii.gz

sub-NDARAC853CR6_ses-HBNsiteCUNY_dir-AP_acq-dwi_epi.nii.gz becomes:
 sub-NDARAC853CR6_ses-HBNsiteCUNY_acq-dwi_dir-AP_epi.nii.gz

sub-NDARAC853CR6_ses-HBNsiteCUNY_dir-PA_acq-dwi_epi.nii.gz becomes:
 sub-NDARAC853CR6_ses

Execute!

In [28]:
bids = fi['info']['BIDS']

print(bids['Filename'])
bids['Filename'] = 'sub-NDARAW247CCF_ses-HBNsiteRU_acq-dwi_dir-PA_epi.nii.gz'
print(bids)

acq.update_file_info(fi.name, {'BIDS': bids})

sub-NDARAW247CCF_ses-HBNsiteRU_acq-dwi_dir-PA_epi.nii.gz
{'template': 'anat_file', 'Filename': 'sub-NDARAW247CCF_ses-HBNsiteRU_acq-dwi_dir-PA_epi.nii.gz', 'Folder': 'anat', 'Path': 'sub-NDARAW247CCF/ses-HBNsiteRU/anat', 'Acq': 'HCP', 'Ce': '', 'Rec': '', 'Run': '', 'Mod': '', 'Modality': 'T1w', 'ignore': False}


{'modified': 1}

In [29]:
for s in subs:
    
    sess = s.sessions()[0]
    
    for acq in sess.acquisitions():
        
        fi = acq.files[0]
        
        if 'epi' in fi.name:
            acq = client.get(acq.id)
            
            new_name = reorder_kvs(fi.name)
            
            print(fi.name, 'becomes:\n', new_name)
            bids = fi['info']['BIDS']

            #print(bids['Filename'])
            bids['Filename'] = new_name
            #print(bids)

            mod = acq.update_file_info(fi.name, {'BIDS': bids})
            
            if mod:
                print("Success!")
            else:
                print("Not changed!")
            print()

sub-NDARAL897CYV_ses-HBNsiteCUNY_dir-AP_acq-fMRI_epi.nii.gz becomes:
 sub-NDARAL897CYV_ses-HBNsiteCUNY_acq-fMRI_dir-AP_epi.nii.gz
Success!

sub-NDARAL897CYV_ses-HBNsiteCUNY_dir-PA_acq-fMRI_epi.nii.gz becomes:
 sub-NDARAL897CYV_ses-HBNsiteCUNY_acq-fMRI_dir-PA_epi.nii.gz
Success!

sub-NDARAL897CYV_ses-HBNsiteCUNY_dir-AP_acq-dwi_epi.nii.gz becomes:
 sub-NDARAL897CYV_ses-HBNsiteCUNY_acq-dwi_dir-AP_epi.nii.gz
Success!

sub-NDARAL897CYV_ses-HBNsiteCUNY_dir-PA_acq-dwi_epi.nii.gz becomes:
 sub-NDARAL897CYV_ses-HBNsiteCUNY_acq-dwi_dir-PA_epi.nii.gz
Success!

sub-NDARAC853CR6_ses-HBNsiteCUNY_dir-AP_acq-fMRI_epi.nii.gz becomes:
 sub-NDARAC853CR6_ses-HBNsiteCUNY_acq-fMRI_dir-AP_epi.nii.gz
Success!

sub-NDARAC853CR6_ses-HBNsiteCUNY_dir-PA_acq-fMRI_epi.nii.gz becomes:
 sub-NDARAC853CR6_ses-HBNsiteCUNY_acq-fMRI_dir-PA_epi.nii.gz
Success!

sub-NDARAC853CR6_ses-HBNsiteCUNY_dir-AP_acq-dwi_epi.nii.gz becomes:
 sub-NDARAC853CR6_ses-HBNsiteCUNY_acq-dwi_dir-AP_epi.nii.gz
Success!

sub-NDARAC853CR6_ses-HBNsi